# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
pip install cartopy geoviews pyproj


Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,282.22,91,100,12.10,TF,1707957583
1,1,oodweyne,9.4092,45.0640,293.06,44,0,3.42,SO,1707957583
2,2,qaqortoq,60.7167,-46.0333,272.56,98,100,2.37,GL,1707957583
3,3,port lincoln,-34.7333,135.8667,291.56,58,12,5.18,AU,1707957379
4,4,daru,-9.0763,143.2092,304.06,66,99,2.46,PG,1707957583


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

import hvplot.pandas

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles='OSM', 
    frame_width=700, 
    frame_height=500, 
    color='City',
    alpha=0.5,
    s='Humidity',
    scale=0.8
)

# Display the map
plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,282.22,91,100,12.10,TF,1707957583
1,1,oodweyne,9.4092,45.0640,293.06,44,0,3.42,SO,1707957583
2,2,qaqortoq,60.7167,-46.0333,272.56,98,100,2.37,GL,1707957583
3,3,port lincoln,-34.7333,135.8667,291.56,58,12,5.18,AU,1707957379
4,4,daru,-9.0763,143.2092,304.06,66,99,2.46,PG,1707957583
...,...,...,...,...,...,...,...,...,...,...
559,559,sivasli,38.4994,29.6836,275.81,99,100,1.67,TR,1707957646
560,560,ardanuc,41.1274,42.0629,282.14,89,100,2.16,TR,1707957646
561,561,gizo,-8.1030,156.8419,302.07,76,50,5.41,SB,1707957646
562,562,columbus,39.9612,-82.9988,276.44,56,20,2.57,US,1707957426


In [6]:
# Create a copy of the DataFrame
filtered_cities_df = city_data_df.copy()

# Narrow down cities that fit criteria (humidity greater than 50)
filtered_cities_df = filtered_cities_df[filtered_cities_df['Humidity'] > 50]

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data
filtered_cities_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,282.22,91,100,12.10,TF,1707957583
2,2,qaqortoq,60.7167,-46.0333,272.56,98,100,2.37,GL,1707957583
3,3,port lincoln,-34.7333,135.8667,291.56,58,12,5.18,AU,1707957379
4,4,daru,-9.0763,143.2092,304.06,66,99,2.46,PG,1707957583
5,5,daimiel,39.0700,-3.6150,285.39,93,100,1.69,ES,1707957583


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,91,
1,oodweyne,SO,9.4092,45.0640,44,
2,qaqortoq,GL,60.7167,-46.0333,98,
3,port lincoln,AU,-34.7333,135.8667,58,
4,daru,PG,-9.0763,143.2092,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 5000  # Specify the radius in meters
params = {
    "radius": radius,
    "categories": "accommodation",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = {"place_type": "city", "city": row["City"]}
    params["bias"] = {"lat": lat, "lon": lng}
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
port-aux-francais - nearest hotel: No hotel found
oodweyne - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
daru - nearest hotel: No hotel found
daimiel - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
fuyang - nearest hotel: No hotel found
dudinka - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
talcahuano - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
tazovsky - nearest hotel: No hotel found
el nido - nearest hotel: No hotel found
nova sintra - nearest hotel: No hotel found
port isabel - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
sal rei - nearest hotel: No hotel found

santa terezinha - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
plastun - nearest hotel: No hotel found
mil'kovo - nearest hotel: No hotel found
mogok - nearest hotel: No hotel found
ulladulla - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
nabire - nearest hotel: No hotel found
trzebnica - nearest hotel: No hotel found
kem' - nearest hotel: No hotel found
futtsu - nearest hotel: No hotel found
jinxiang - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
margarita belen - nearest hotel: No hotel found
dunedin - nearest hotel: No hotel found
san giorgio di nogaro - nearest hotel: No hotel found
chernyshevskiy - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
saguenay - nearest hotel: No hotel found
vreed-en-hoop - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
kuytun - nearest hotel: No hotel found
lexington - nea

aleksandrov - nearest hotel: No hotel found
san matias - nearest hotel: No hotel found
balaipungut - nearest hotel: No hotel found
istra - nearest hotel: No hotel found
idil - nearest hotel: No hotel found
krasnaya gorbatka - nearest hotel: No hotel found
monrovia - nearest hotel: No hotel found
bafq - nearest hotel: No hotel found
severo-yeniseyskiy - nearest hotel: No hotel found
saki - nearest hotel: No hotel found
alappuzha - nearest hotel: No hotel found
kuanda - nearest hotel: No hotel found
puerto deseado - nearest hotel: No hotel found
patamda - nearest hotel: No hotel found
danane - nearest hotel: No hotel found
midoun - nearest hotel: No hotel found
koprivnica - nearest hotel: No hotel found
kingsbridge - nearest hotel: No hotel found
pematangsiantar - nearest hotel: No hotel found
cockburn town - nearest hotel: No hotel found
el calafate - nearest hotel: No hotel found
nizhneudinsk - nearest hotel: No hotel found
glace bay - nearest hotel: No hotel found
santa elena de uaire

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,91,No hotel found
1,oodweyne,SO,9.4092,45.0640,44,No hotel found
2,qaqortoq,GL,60.7167,-46.0333,98,No hotel found
3,port lincoln,AU,-34.7333,135.8667,58,No hotel found
4,daru,PG,-9.0763,143.2092,66,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
pip install hvplot


Note: you may need to restart the kernel to use updated packages.


In [11]:
# Configure the map plot
plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='Hotel Name',
    hover_cols=["City", "Country", "Hotel Name", "Humidity"],
    alpha= 0.5,
    s=50,
    scale= 0.8)

# Display the map
plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country,Humidity)